In [2]:
import pandas as pd
import warnings
warnings.filterwarnings(action = 'ignore')
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)

In [3]:
vod_08 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_VOD_2308.csv', encoding = 'cp949', sep = '\t')
vod_09 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_VOD_2309.csv', encoding = 'cp949', sep = '\t')

In [4]:
# 8,9월 데이터 합치기
vod_89 = pd.concat([vod_08, vod_09], ignore_index=True)
vod_89.head(2)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227


In [5]:
def preprocessing(data):
  df = data.copy()
  
  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0
    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  # 런타임 0분인 것 제거
  df = df[df['disp_rtm'] != 0]

  # 무삭제 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)
  
  # 예고편 제거
  df = df[~df['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]
  
  # 예약구매, 사전구매 제거
  df = df[~df['asset_nm'].str.contains('예약구매|사전구매')]

  # 00회가 포함된 행은 런타임이 0또는 1이므로 제거
  df = df[~df['asset_nm'].str.contains(r'\b00회\b')]
  
  ## 괄호와 그 안의 내용 제거 
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\…', '', regex=True) # … 기호 제거
  df['asset_nm'] = df['asset_nm'].str.replace('-', " ")
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')
  # df['asset_nm'] = df['asset_nm'].str.strip()

  # df3['series_nm'] = df3['asset_nm'].str.replace(r'\d+회$', '', regex=True)
  # df3['series_nm'] = df3['series_nm'].str.replace(r'\d+회\.', '', regex=True) 
  # df3['series_nm'] = df3['series_nm'].str.replace(r'\d+화$', regex = True)

  # use_tms 분 단위로 변경
  df['use_tms'] = round(df['use_tms'] / 60, 1)

  # 셋탑번호가 66056000인 것 삭제 - content 에서 이상치 아이디로 판별
  df = df[df['subsr'] != 66056000]

  return df

In [6]:
df = preprocessing(vod_89)
df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737


In [7]:
# 프로그램 회차별 정보
df1 = df[['asset_nm', 'ct_cl', 'genre_of_ct_cl', 'ACTR_DISP', 'disp_rtm']].drop_duplicates().reset_index(drop = True)
df1.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78


In [8]:
# 유저마다 각 회차를 본 시간을 더함
df2 = pd.DataFrame(df.groupby(['subsr','asset_nm', 'ct_cl', 'genre_of_ct_cl', 'disp_rtm'])['use_tms'].sum()).reset_index()
df2 = df2[(df2['disp_rtm'] != 0) & (df2['use_tms'] != 0)]
df2

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3
...,...,...,...,...,...,...
5705,67148000,타요의 씽씽극장 동요2 10회,키즈,기타,2,2.0
5706,67148000,타요의 씽씽극장 동요2 11회,키즈,기타,2,0.5
5707,67154000,스트릿 우먼 파이터 2 04회,TV 연예/오락,기타,131,1.2
5708,67161000,스파이 코드명 포춘,영화,액션/어드벤쳐,114,0.2


In [9]:
# 시청시간으로 유저가 이 회차를 시청했는지 여부를 결정
## 시청시간을 런타임으로 나눠 0.1 이 넘으면 시청한 것으로 간주
df2['watched'] = df2['use_tms'] / df2['disp_rtm']
df2['watched'] = df2['watched'].apply(lambda x : 1 if x >= 0.2 else 0)
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4,0
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1,1
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6,0
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8,1
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3,0


In [10]:
# 회차 제거
df11 = df1.copy()

df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+화$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.strip()
df11['asset_nm'] = df11['asset_nm'].str.rstrip('.')
df11

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",78
...,...,...,...,...,...
3850,엄마가 화났다,키즈,학습,-,8
3851,핑크퐁 자동차 동화,키즈,학습,핑크퐁,4
3852,간질간질,키즈,학습,-,6
3853,월간 아기상어,키즈,학습,아기상어,1


In [11]:
# 유저 시청 정보의 회차 제거한 변수 추가
df2['series_nm'] = df2['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+회\.', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+화$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.strip()
df2['series_nm'] = df2['series_nm'].str.rstrip('.')
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1,1,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6,0,신성한 이혼
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8,1,신성한 이혼
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3,0,신성한 이혼


In [12]:
df3 = df2[['subsr', 'series_nm', 'ct_cl',	'genre_of_ct_cl', 'watched']].copy()
df3.head()

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
0,59879000,소방서 옆 경찰서,TV드라마,기타,0
1,59879000,소방서 옆 경찰서,TV드라마,기타,1
2,59879000,신성한 이혼,TV드라마,기타,0
3,59879000,신성한 이혼,TV드라마,기타,1
4,59879000,신성한 이혼,TV드라마,기타,0


In [13]:
# 유저의 시리즈별 시청 횟수
df4 = df3.groupby(['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl'])['watched'].sum().reset_index()
df4 = df4[df4['watched']!= 0]
df4

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
0,59879000,소방서 옆 경찰서,TV드라마,기타,1
1,59879000,신성한 이혼,TV드라마,기타,3
2,59895000,금이야 옥이야,TV드라마,기타,1
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,1
4,59900000,그것이알고싶다,TV 시사/교양,기타,3
...,...,...,...,...,...
2066,67140000,경남 통영 2부,우리동네,연예/오락,1
2067,67140000,밀수,영화,액션/어드벤쳐,1
2068,67140000,잠자는 숲속의 공주,키즈,기타,1
2070,67148000,타요의 씽씽극장 동요2,키즈,기타,11


In [14]:
# 8~9월 시리즈별 시청된 총 횟수
df5 = pd.DataFrame(df11[['asset_nm', 'ct_cl', 'genre_of_ct_cl']].value_counts().reset_index())
df5.columns = ['series_nm', 'ct_cl', 'genre_of_ct_cl', 'watched_all']
df5

,series_nm,ct_cl,genre_of_ct_cl,watched_all
0,금이야 옥이야,TV드라마,기타,83
1,연희공략: 건륭황제의여인,TV드라마,외화 시리즈,63
2,TV소설 은희,TV드라마,기타,63
3,인간극장,TV 시사/교양,기타,55
4,런닝맨,TV 연예/오락,기타,54
...,...,...,...,...
1019,봉신연의,영화,액션/어드벤쳐,1
1020,부산,영화,액션/어드벤쳐,1
1021,북 오브 러브,영화,멜로,1
1022,분노의 질주: 라이드 오어 다이,영화,액션/어드벤쳐,1


In [15]:
df6 = df4.merge(df5, on = ['series_nm', 'ct_cl', 'genre_of_ct_cl'], how = 'left')
df6

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all
0,59879000,소방서 옆 경찰서,TV드라마,기타,1,11
1,59879000,신성한 이혼,TV드라마,기타,3,12
2,59895000,금이야 옥이야,TV드라마,기타,1,83
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3
4,59900000,그것이알고싶다,TV 시사/교양,기타,3,21
...,...,...,...,...,...,...
1494,67140000,경남 통영 2부,우리동네,연예/오락,1,1
1495,67140000,밀수,영화,액션/어드벤쳐,1,2
1496,67140000,잠자는 숲속의 공주,키즈,기타,1,1
1497,67148000,타요의 씽씽극장 동요2,키즈,기타,11,13


In [16]:
# 유저의 프로그램 총 시청 횟수
df7 = df6.groupby(['subsr'])['watched'].sum().reset_index()
df7.columns = ['subsr', 'watched_cnt']
df7

,subsr,watched_cnt
0,59879000,4
1,59895000,1
2,59900000,8
3,59921000,2
4,59930000,12
...,...,...
327,67117000,10
328,67129000,1
329,67140000,3
330,67148000,11


In [47]:
df8 = df6.merge(df7, on = 'subsr', how = 'left')
df8 = df8[df8['watched_cnt'] != 0].reset_index(drop = True)
df8

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt
0,59879000,소방서 옆 경찰서,TV드라마,기타,1,11,4
1,59879000,신성한 이혼,TV드라마,기타,3,12,4
2,59895000,금이야 옥이야,TV드라마,기타,1,83,1
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3,8
4,59900000,그것이알고싶다,TV 시사/교양,기타,3,21,8
...,...,...,...,...,...,...,...
1494,67140000,경남 통영 2부,우리동네,연예/오락,1,1,3
1495,67140000,밀수,영화,액션/어드벤쳐,1,2,3
1496,67140000,잠자는 숲속의 공주,키즈,기타,1,1,3
1497,67148000,타요의 씽씽극장 동요2,키즈,기타,11,13,11


In [18]:
# 시청횟수가 5개 이상인 유저만 
user_cnt = df8['subsr'].value_counts()
filter_users = user_cnt[user_cnt >= 3].index
df8 = df8[df8['subsr'].isin(filter_users)]
df8

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3,8
4,59900000,그것이알고싶다,TV 시사/교양,기타,3,21,8
5,59900000,마파도,영화,코미디,1,1,8
6,59900000,범죄도시3,영화,액션/어드벤쳐,1,1,8
7,59900000,초대: 스와핑 데이,영화,멜로,1,1,8
...,...,...,...,...,...,...,...
1481,67055000,비밀의 여자,TV드라마,기타,1,7,88
1482,67055000,소방서 옆 경찰서 그리고 국과수,TV드라마,기타,6,12,88
1483,67055000,신발 벗고 돌싱포맨,TV 연예/오락,기타,1,10,88
1484,67055000,실화탐사대,TV 시사/교양,기타,3,9,88


In [18]:
# 시청 여부, 해당 시리즈 총 횟수, 유저의 프로그램 총 시청 횟수를 이용해
# 유저의 프로그램 선호도를 측정
import numpy as np
def scoring(df):
  N = df['watched_all'] # 해당 프로그램 전체 회차 수
  L = df['watched_cnt'] # 유저의 총 시청 프로그램 회차 수
  n = df['watched'] # 유저의 해당 프로그램 시청 수
  lam = np.log(2) / 2
  w1 = 1 - np.exp(-1 * lam * N)
  data = pd.DataFrame([N, L]).T
  def custom_weight(data):
    if data['watched_cnt'] < data['watched_all']:
      return data['watched_all'] / data['watched_cnt']
    else:
      return 1

  w2 = data.apply(custom_weight, axis = 1)

  score = (n / N) * w1 * w2

  return score

In [20]:
df8['score'] = scoring(df8)
df8

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt,score
0,59879000,소방서 옆 경찰서,TV드라마,기타,1,11,4,0.244476
1,59879000,신성한 이혼,TV드라마,기타,3,12,4,0.738281
2,59895000,금이야 옥이야,TV드라마,기타,1,83,1,1.000000
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3,8,0.215482
4,59900000,그것이알고싶다,TV 시사/교양,기타,3,21,8,0.374741
...,...,...,...,...,...,...,...,...
1494,67140000,경남 통영 2부,우리동네,연예/오락,1,1,3,0.292893
1495,67140000,밀수,영화,액션/어드벤쳐,1,2,3,0.250000
1496,67140000,잠자는 숲속의 공주,키즈,기타,1,1,3,0.292893
1497,67148000,타요의 씽씽극장 동요2,키즈,기타,11,13,11,0.988951


In [21]:
final_df = df8[['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl', 'score']]
final_df.columns = ['userid', 'program', 'main_cat', 'sub_cat', 'score']
final_df = final_df.sort_values(by = 'userid').reset_index(drop = True)
final_df['category'] = final_df['main_cat'].apply(lambda x : x if x in ['영화', '키즈'] else 'TV프로그램')
final_df

,userid,program,main_cat,sub_cat,score,category
0,59879000,소방서 옆 경찰서,TV드라마,기타,0.244476,TV프로그램
1,59879000,신성한 이혼,TV드라마,기타,0.738281,TV프로그램
2,59895000,금이야 옥이야,TV드라마,기타,1.000000,TV프로그램
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램
4,59900000,그것이알고싶다,TV 시사/교양,기타,0.374741,TV프로그램
...,...,...,...,...,...,...
1494,67140000,경남 통영 2부,우리동네,연예/오락,0.292893,TV프로그램
1495,67140000,밀수,영화,액션/어드벤쳐,0.250000,영화
1496,67140000,잠자는 숲속의 공주,키즈,기타,0.292893,키즈
1497,67148000,타요의 씽씽극장 동요2,키즈,기타,0.988951,키즈


In [22]:
final_df['rename'] = final_df['program'].apply(lambda x : x.replace(' ', '') if isinstance(x, str) else x)
final_df

,userid,program,main_cat,sub_cat,score,category,rename
0,59879000,소방서 옆 경찰서,TV드라마,기타,0.244476,TV프로그램,소방서옆경찰서
1,59879000,신성한 이혼,TV드라마,기타,0.738281,TV프로그램,신성한이혼
2,59895000,금이야 옥이야,TV드라마,기타,1.000000,TV프로그램,금이야옥이야
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램,2022역사저널그날
4,59900000,그것이알고싶다,TV 시사/교양,기타,0.374741,TV프로그램,그것이알고싶다
...,...,...,...,...,...,...,...
1494,67140000,경남 통영 2부,우리동네,연예/오락,0.292893,TV프로그램,경남통영2부
1495,67140000,밀수,영화,액션/어드벤쳐,0.250000,영화,밀수
1496,67140000,잠자는 숲속의 공주,키즈,기타,0.292893,키즈,잠자는숲속의공주
1497,67148000,타요의 씽씽극장 동요2,키즈,기타,0.988951,키즈,타요의씽씽극장동요2


In [23]:
final_df.to_csv('../data/vod89.csv', index = 0)

In [46]:
# testdata 중에서 vod에 없는 유저만 추출
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
train_user = set(final_df.userid.unique())
test_user = set(testdata.subsr.unique())
novod_user = list(test_user - train_user)

testdata = testdata[testdata.subsr.isin(novod_user)]
testdata.to_csv('../data/novod_user.csv', index = 0)